In [3]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0
scikit-learn-intelex==2021.20220215.212715


In [17]:
!python -V

Python 3.9.12


In [27]:
import pickle
import pandas as pd
import numpy as np
import os

In [19]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [20]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [21]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [22]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [23]:
# Calculate the standard deviation of the predicted duration
std_dev = np.std(y_pred)
print(f"Standard deviation of predicted duration: {std_dev:.2f}")

Standard deviation of predicted duration: 6.25


Q2

In [32]:
# Create a dataframe with ride_id and predictions
df_result = pd.DataFrame()
df_result['ride_id'] = '2023/03_' + df.index.astype('str')
df_result['predictions'] = y_pred

# Save the result as parquet
output_file = 'results.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

# file size
file_size = os.path.getsize(output_file) / (1024 * 1024)  # Convert to MB
print(f"Size of the output file: {file_size:.2f}M")

Size of the output file: 65.46M


Q4

In [2]:
pip install pipenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pipenv install numpy pandas scikit-learn==1.5.0 pyarrow

Creating a virtualenv for this project
Pipfile: /workspaces/mlops-zoomcamp/04-deployment/Pipfile
Using default python from /home/codespace/anaconda3/bin/python3.9.12 to create 
virtualenv...
⠇ Creating virtual environment.....created virtual environment CPython3.9.12.final.0-64 in 643ms
  creator 
CPython3Posix(dest=/home/codespace/.local/share/virtualenvs/04-deployment-yWDDY4
m_, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, 
wheel=bundle, via=copy, app_data_dir=/home/codespace/.local/share/virtualenv)
    added seed packages: pip==24.3.1, setuptools==75.2.0, wheel==0.44.0
  activators 
BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator
,PythonActivator

✔ Successfully created virtual environment!
⠇ Creating virtual environment...
Virtualenv location: /home/codespace/.local/share/virtualenvs/04-deployment-yWDDY4m_
Creating a Pipfile for this project...
Pipfile.lock not found, creating..

Q5

In [6]:
%run score_params.py --year 2023 --month 4

Mean predicted duration: 14.29
